## Spatial queries in duckdb using jinja and magic_duckdb


#### Links: <br> https://pypi.org/project/magic-duckdb/<br> 
https://duckdb.org/<br>
#### GitHub:<br>
https://github.com/RandomFractals/geo-data-viewer/tree/master/data/csv
https://github.com/Tor-Storli/Geospatial_Data/tree/main/data<br>
#### Tor-Storli-YouTube:<br>
https://youtu.be/hMP-LGju1IA<br>
https://youtu.be/zoUcuJbGCp0

#### If needed you can change the fonts like this

In [ ]:
%%HTML
<style>
    body {
        --vscode-font-family: "Arial";
    }
</style>

#### Import Python Libraries and add a reference to the magic_duckdb package

In [ ]:
import duckdb as duckdb
import magic_duckdb
import pandas as pd
import requests
from lxml import html

%load_ext magic_duckdb

#### Install and load the duckdb spatial and httpfs extensions

In [ ]:
%%dql 
-- INSTALL spatial;
-- INSTALL httpfs;
LOAD spatial;
LOAD httpfs;

#### Query data from GeoJson file over the internet

In [ ]:
%%time
%%dql -t show
SELECT * FROM 'https://raw.githubusercontent.com/Tor-Storli/Geospatial_Data/main/data/IL_Counties_Census.json' LIMIT 5;

#### Show the your current working directory

In [ ]:
%pwd

#### Create a local GeoJson file from the GeoJson file located on Github

In [ ]:
%%time
%%dql
COPY (SELECT * FROM 'https://raw.githubusercontent.com/Tor-Storli/Geospatial_Data/main/data/IL_Counties_Census.json')
      TO 'IL_Counties_Census.json' (FORMAT 'json');

### Using jinja2 python library in magic_duckdb

## JINJA Magic

#### We can use jinja2 text templating language library to make dynamic queries. <br> You can find out more about it here: <br> https://codeburst.io/jinja-2-explained-in-5-minutes-88548486834e  <br> https://jinja.palletsprojects.com/en/2.11.x/templates/

### Some examples: <br> ### 

#### Create some sample data and load into a pandas dataframe ####

In [ ]:
# Sample data (replace with your actual data)
data = {
    'dates': ['2024-03-12', '2024-03-13','2024-03-14', '2024-03-15', '2024-03-16'],
    'distances': ['5K', '3K', '3K', '3K', '5k']
}
df_Runs = pd.DataFrame(data)

#### Basic dynamic sort query

In [ ]:
sort_column = 'dates'
sort_direction = 'Asc'
%dql --jinja2 SELECT * FROM df_Runs ORDER BY {{ sort_column }} {{ sort_direction }};

#### Let us inspect the datatypes in our dataframe

In [ ]:
df_Runs.dtypes

#### Convert dates column to a timestamp datatype

In [ ]:
df_Runs['dates'] = pd.to_datetime(df_Runs['dates'])

In [ ]:
df_Runs.dtypes

#### Create a pivot query using jinja2

##### Explanation:
- We use a `{%- for run_type in run_types %}` loop to iterate through the list of run types types.
- For each run type, we create a conditional aggregation using a CASE statement:<br>`SUM(CASE WHEN distances = '{{ run_type }}' THEN 1 ELSE 0 END`.
- The `tot_{{ distance }}` alias represents the total count for that run distance type.
- The `GROUP BY 1` groups the results by the first column in our query - the `dates` column.
- Finally, we order the results by date in ascending order.

In [ ]:
# Generate the SQL query dynamically using Jinja2-like template
run_types = ['5K', '3K']

jinja2_pivot_query = """
SELECT dates,
    {%- for run_type in run_types %}
    SUM(CASE WHEN distances = '{{ run_type }}' THEN 1 ELSE 0 END) AS tot_{{ run_type }}
    {%- if not loop.last -%}, {% endif -%}
    {%- endfor %}
FROM df_Runs
GROUP BY 1
ORDER BY 1 ASC;
"""
print(jinja2_pivot_query)

#### Copy the query from above and run it using magic_duckdb and jinja2

In [ ]:
%%dql --jinja2 -t df

SELECT dates,
    {%- for run_type in run_types %}
    SUM(CASE WHEN distances = '{{ run_type }}' THEN 1 ELSE 0 END) AS tot_{{ run_type }}
    {%- if not loop.last -%}, {% endif -%}
    {%- endfor %}
FROM df_Runs
GROUP BY 1
ORDER BY 1 ASC;


#### We can also extract the week from the dates so we calculate the total distances we ran that week

In [ ]:
%%dql --jinja2 -t df
SELECT date_part('week', dates) as week,
    {%- for run_type in run_types %}
    SUM(CASE WHEN distances = '{{ run_type }}' THEN 1 ELSE 0 END) AS tot_{{ run_type }} 
    {%- if not loop.last -%}, {% endif -%}
    {%- endfor %}
FROM df_Runs
GROUP BY 1
ORDER BY 1 ASC;

#### Using regular duckdb Query

In [ ]:
%%dql -t df
SELECT dates,
    SUM(CASE WHEN distances = '5K' THEN 1 ELSE 0 END) AS num_5K,
    SUM(CASE WHEN distances = '3K' THEN 1 ELSE 0 END) AS num_3K
FROM df_Runs
GROUP BY dates;

#### The problem starts when we add more distances. <br>We then have to keep adding distances to the queries

### So let us create a more elegant way of running queries dynamically,<br> we use pandas, jinja2, and magic_duckdb

In [ ]:
# Sample data (replace with your actual data)
data = {
    'dates': ['2024-03-12', '2024-03-13','2024-03-14', 
              '2024-03-15', '2024-03-16','2024-03-17',
              '2024-03-18', '2024-03-19', '2024-03-20'],
    'distances': ['5K', '3K', '3K', '3K', '5K', '7K','10K', '7K','5K']
}
df_Runs = pd.DataFrame(data)
df_Runs['dates'] = pd.to_datetime(df_Runs['dates'])
df_Runs

#### Create a dynamic run_type list containing unique distance values.

In [ ]:
run_types =  df_Runs['distances'].unique()
run_types

In [ ]:
%%dql --jinja2 -t df
SELECT dates,
    {%- for run_type in run_types %}
    SUM(CASE WHEN distances = '{{ run_type }}' THEN 1 ELSE 0 END) AS tot_{{ run_type }}
    {%- if not loop.last -%}, {% endif -%}
    {%- endfor %}
FROM df_Runs
GROUP BY 1
ORDER BY 1 ASC;

#### Total each distance for the two weeks.

In [ ]:
%%dql --jinja2 -t df
SELECT date_part('week', dates) as week,
    {%- for run_type in run_types %}
    SUM(CASE WHEN distances = '{{ run_type }}' THEN 1 ELSE 0 END) AS tot_{{ run_type }}
    {%- if not loop.last -%}, {% endif -%}
    {%- endfor %}
FROM df_Runs
GROUP BY 1
ORDER BY 1 ASC;

## Let us use jinja and retrieve a GeoJson file from Github and save a copy to our current local drive

In [ ]:
baseurl = "https://raw.githubusercontent.com/Tor-Storli/Geospatial_Data/main/data/"
cafile = "CA_Counties_Census.json"
jsonformat = "JSON"
csvformat = "CSV, DELIMITER '|', HEADER"
print()
%dql --jinja2 COPY (SELECT * FROM '{{baseurl}}{{cafile}}') TO '{{cafile}}' (FORMAT {{jsonformat}});

#### Delete all `json` and `csv` files that are in the current working directory

In [ ]:
import os
currdir = os.getcwd() 

filelist = [ f for f in os.listdir(currdir) if f.endswith(".json") or f.endswith(".csv") ]
for f in filelist:
    os.remove(os.path.join(currdir, f))
    print('deleting file: ' + f)

### Let us use jinja and retrieve several GeoJson files and a csv file from Github<br> and save a copies to our current local drive

In [ ]:
%%time

# Create a utility function that returns the format
def get_format(filename):
    extension = file.split('.')[1]
    jsonformat = "JSON"
    csvformat = "CSV, DELIMITER '|', HEADER"
    if extension == "json":
        return jsonformat
    if extension == "csv":
        return csvformat
    
# Set up file list for download
baseurl = "https://raw.githubusercontent.com/Tor-Storli/Geospatial_Data/main/data/"
files = ["CA_Counties_Census.json", "IL_Counties_Census.json", "us_food_restaurants.json", "targets.csv"]
#files = ["targets.csv"]
jsonformat = "JSON"
csvformat = "CSV, DELIMITER '|', HEADER"
    
    
# iterate file list using for loop
for file in files:
    fileformat = get_format(file)
    %dql --jinja2 COPY (SELECT * FROM '{{baseurl}}{{file}}') TO '{{file}}' (FORMAT {{fileformat}});

## Visualize spatial data with Keppler Map View Extension

### Visualize the Counties spatial data in Map View 

#### Let us download airport data from GitHub and save it locally as a csv file. 

In [ ]:
import requests
#import pandas as pd
#from lxml import html

# Define the Wikipedia URL
url = "https://en.wikipedia.org/wiki/List_of_airports_in_Illinois"

# Define the XPath expression for the table
xpath_expression = '//*[@id="mw-content-text"]/div[1]/table'
#xpath_expression = '//*[@id="mw-content-text"]/div[1]/table/tbody'

# Send an HTTP request to the Wikipedia page
response = requests.get(url)

# Parse the HTML content using lxml
tree = html.fromstring(response.content)

# Find the table using the XPath expression
table = tree.xpath(xpath_expression)[0]

#print(table)
# Convert the table to a Pandas DataFrame
df_wiki = pd.read_html(html.tostring(table))[0]

df_wiki.dropna(inplace=True)

# remove all records that do not have FAA numbers
#df_csv = df_wiki.drop(index=1)

#print(df_wiki.head())
# Save the DataFrame to a CSV file
df_wiki.to_csv("wiki_il_airport_data.csv", index=False)

# Print a success message
print("Illinois airport data from wikipedia has been downloaded and saved as wiki_il_airport_data.csv")


### Create a data frame from the GeoJson file

In [ ]:
import json

# Read the JSON file
with open("usa-airports.map.json", "r") as json_file:
    data = json.load(json_file)

# Extract the "allData" array
all_data = data["datasets"][0]["data"]["allData"]

# Define column names and data types
columns = ["iata", "name", "city", "state", "country", "latitude", "longitude"]
data_types = {
    "iata": "str",
    "name": "str",
    "city": "str",
    "state": "str",
    "country": "str",
    "latitude": "float",
    "longitude": "float"
}

# Create a Pandas DataFrame
df_usair = pd.DataFrame(all_data, columns=columns)
df_usair.dropna(inplace=True)

# Convert data types
for col, dtype in data_types.items():
    df_usair[col] = df_usair[col].astype(dtype)


In [ ]:
%%dql -t df -o df_wiki
SELECT "City served" as City,
        FAA,
        IATA,
        "Airport name" As Airport,
        Role,
        "Enplanements (2019)" As Passengers
FROM read_csv_auto('wiki_il_airport_data.csv', Header=TRUE);

In [ ]:
merged_df = pd.merge(df_usair, df_wiki, left_on='iata', right_on='FAA')

merged_df.drop(columns=['City'], inplace=True)
merged_df

In [ ]:
%%dql
CREATE OR REPLACE TABLE WIKI_US_AIRPORTS AS 
SELECT iata,name,city,state,country,FAA,Airport,Role,Passengers, 
       ST_Point(longitude, latitude) AS Geometry
FROM merged_df;     

In [ ]:
%%dql 
COPY WIKI_US_AIRPORTS TO 'wiki_us_airports.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJson') 

#### Let us create a new table based on the California Counties GeoJson file. <br>We can use the flexible `ST_Read()` function

In [ ]:
%%dql
CREATE OR REPLACE TABLE CA_Counties AS SELECT * FROM ST_Read('CA_Counties_Census.json');

#### Let us use the `ST_Union_Agg()` aggregate function and combine a few counties into one shape.

In [ ]:
%%dql
CREATE OR REPLACE TABLE CA_Counties_Agg AS
SELECT ST_Union_Agg("geom") as geoAgg FROM 'CA_Counties' 
WHERE id in(6015, 6093,6049,4445,6023,6105,6089,6035)
GROUP BY id;

#### Let us now save the new aggregated shape file to our local folder and view it on a map.

In [ ]:
%%dql
COPY CA_Counties_Agg TO 'CA_Counties_Agg.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJson') 

### END OF FILE